In [1]:
import pickle

## Load pickle
with open("soft_dedup_ngramdf.pkl","rb") as f:
    soft_dedup_ngramdf = pickle.load(f)

In [2]:
soft_dedup_ngramdf.sort_values(by=["num_of_exploit"], ascending=False).head(10)

,N,Ngram_syscall,num_of_exploit,EIDs
459,2,"[12, 12]",55,"[10613, 12, 1397, 14814, 15023, 15024, 15285, ..."
455,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
399,2,"[0, 3]",38,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
338,2,"[13, 14]",37,"[1397, 14814, 2004, 2006, 2031, 24746, 3, 3134..."
396,2,"[0, 5]",35,"[12, 1397, 14814, 15023, 15024, 15285, 15704, ..."
443,2,"[61, 13]",33,"[1397, 14814, 15944, 18411, 2004, 2006, 2031, ..."
354,2,"[13, 13]",33,"[1397, 14814, 2004, 2006, 2031, 22362, 22363, ..."
335,2,"[5, 0]",32,"[12, 1397, 14814, 15023, 15024, 15285, 15704, ..."
81,12,"[13, 13, 14, 9, 14, 56, 11, 14, 61, 13, 13, 14]",29,"[1397, 14814, 2004, 2006, 2031, 24746, 3, 3134..."
243,3,"[12, 12, 257]",29,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."


In [3]:
# 모든 익스플로잇 path laod
import subprocess
import json

pathFile = subprocess.check_output(f"find /opt/output/perm/path/*.json", shell=True).decode().strip().split()
EIDList = list(map(lambda x: x.replace("/opt/output/perm/path/","").replace(".json",""), pathFile))
pathList = dict() # 익스플로잇 코드 path dictionary의 dictionary - pathList[EID][path] = path 파일 정보
for EID in EIDList:
    with open(f"/opt/output/perm/path/{EID}.json", "r") as f:
        pathList[EID] = dict()
        pathList[EID]["path"] = json.load(f)
        
# dict {EID: all path list} 만들어 주기
eid_all_path_dict = dict()
for EID, pathset in pathList.items():
    eid_all_path_dict[EID] = list()
    for userfunc, pathset in pathset['path'].items():
        if not len(pathset) == 0:
            eid_all_path_dict[EID].extend(pathset)
            
# dict {EID: syscall set} 만들어 주기
eid_syscall_set_dict = dict()
for EID, pathList in eid_all_path_dict.items():
    eid_syscall_set_dict[EID] = set()
    for path in pathList:
        for syscall in path:
            eid_syscall_set_dict[EID].add(syscall)

In [4]:
# syscall 바꿔주기

for EID, pathSet in eid_syscall_set_dict.items():
    if "setns" in pathSet:
        pathSet = pathSet - {'setns'} | {'308'}
    if "time" in pathSet:
        pathSet = pathSet - {'time'} | {'201'}
    if "capset" in pathSet:
        pathSet = pathSet - {'capset'} | {'126'}
    if "fstat" in pathSet:
        pathSet = pathSet - {'fstat'} | {'5'}
    if "uname" in pathSet:
        pathSet = pathSet - {'uname'} | {'63'}
    if "__NR_vm86old" in pathSet:
        pathSet = pathSet - {'__NR_vm86old'} | {'436'}
    if "__NR_socketcall" in pathSet:
        pathSet = pathSet - {'__NR_socketcall'} | {'437'}
    if "sigaction" in pathSet:
        pathSet = pathSet - {'sigaction'} | {'13'}
    if "ioctl" in pathSet:
        pathSet = pathSet - {'ioctl'} | {'16'}
    if "kmalloc" in pathSet:
        pathSet = pathSet - {'kmalloc'} | {'12'}
    if "shutdown" in pathSet:
        pathSet = pathSet - {'shutdown'} | {'48'}
    if "msgrcv" in pathSet:
        pathSet = pathSet - {'msgrcv'} | {'70'}
    if "ioctl" in pathSet:
        pathSet = pathSet - {'ioctl'} | {'16'}
    eid_syscall_set_dict[EID] = pathSet

In [5]:
# syscall 아닌 문자 제거하기

eid_only_syscall_set_dict = dict()
for EID, pathset in eid_syscall_set_dict.items():
    eid_only_syscall_set_dict[EID] = set()
    for syscall in list(pathset):
        if syscall.isdigit():
            eid_only_syscall_set_dict[EID].add(syscall)

# 4.1 개별 시스템콜 기반 필터링 메커니즘 평가

## 악성코드가 사용하는 시스템콜 개수

In [6]:
syscallset = set()
for EID, pathset in eid_only_syscall_set_dict.items():
    syscallset = syscallset | pathset
    
syscallset = syscallset - {'1073742361'}
len(syscallset)

140

## 정상동작 어플리케이션이 사용하는 시스템콜 개수

In [7]:
import pickle

# Load data
with open('b_sysSet.pkl', 'rb') as handle:
    b_sysSet = pickle.load(handle)

In [8]:
imgList = {"httpd", "mongodb", "mysql", "nginx", "redis", "mariadb", "node", "tomcat"}
imgList = {"httpd", "mongodb", "mysql", "nginx", "redis", "mariadb", "node", "tomcat", "gcc", "openjdk", "gzip", "bzip2", "ghostscript", "lowriter", "qalc"}


b_sysSet_total = set()
for imgName in imgList:
    print(f"image {imgName} syscall set #: {len(b_sysSet[imgName])}")
    b_sysSet_total = b_sysSet_total | b_sysSet[imgName]
print(f"total syscall set #: {len(b_sysSet_total)}")

image httpd syscall set #: 21
image bzip2 syscall set #: 17
image node syscall set #: 13
image mariadb syscall set #: 22
image ghostscript syscall set #: 38
image qalc syscall set #: 42
image redis syscall set #: 14
image gzip syscall set #: 20
image nginx syscall set #: 14
image gcc syscall set #: 38
image openjdk syscall set #: 57
image mongodb syscall set #: 45
image tomcat syscall set #: 25
image lowriter syscall set #: 76
image mysql syscall set #: 39
total syscall set #: 107


In [9]:
b_sysSet_total = b_sysSet_total | {'272','13','14','15','22','33','49','50','53','59','62','63','68','69','70','102','103','104','105','106','107','108','109','110','112','113','117','118','119','158','160','203','213','218','437'}
len(b_sysSet_total)

125

## 악성 - 양성 = 개별 시스템콜 정책

In [10]:
individual_policy = set()
individual_policy = syscallset - b_sysSet_total
len(individual_policy)

42

In [11]:
print(individual_policy)

{'141', '292', '247', '317', '261', '248', '254', '144', '283', '293', '101', '91', '244', '26', '140', '27', '161', '149', '125', '201', '275', '80', '436', '294', '278', '135', '250', '71', '321', '116', '126', '29', '298', '308', '154', '166', '134', '73', '37', '34', '165', '30'}


## 정책을 통해 완화되는 악성코드 조사

In [12]:
individual_mitigate_exploit = set()
for policy in list(individual_policy):
    EID_cnt = 0
    for EID, syscallset in eid_only_syscall_set_dict.items():
        if policy in syscallset:
            individual_mitigate_exploit = individual_mitigate_exploit | {EID}
            EID_cnt = EID_cnt + 1
            
len(individual_mitigate_exploit)

67

# 4.2 시스템콜 시퀀스 기반 필터링 메커니즘 평가

## 길이별 N-gram 시스템콜 시퀀스의 개수 및 관련 악성코드의 개수

### 악성코드 N-gram

In [13]:
# 모든 익스플로잇 path laod
import re
import subprocess
import json

pathFile = subprocess.check_output(f"find /opt/output/perm/path/*.json", shell=True).decode().strip().split()
EIDList = list(map(lambda x: x.replace("/opt/output/perm/path/","").replace(".json",""), pathFile))
pathList = dict() # 익스플로잇 코드 path dictionary의 dictionary - pathList[EID][path] = path 파일 정보
for EID in EIDList:
    with open(f"/opt/output/perm/path/{EID}.json", "r") as f:
        pathList[EID] = dict()
        pathList[EID]["path"] = json.load(f)
        
# dict {EID: all path list} 만들어 주기
eid_all_path_dict = dict()
for EID, pathset in pathList.items():
    eid_all_path_dict[EID] = list()
    for userfunc, pathset in pathset['path'].items():
        if not len(pathset) == 0:
            eid_all_path_dict[EID].extend(pathset)

# path 다듬기
for EID, pathList in eid_all_path_dict.items():
    new_pathList = list()
    for path in pathList:
        if "setns" in path:
            str_path = ",".join(path)
            path = str_path.replace('setns','308').split(',')
        if "time" in path:
            str_path = ",".join(path)
            path = str_path.replace('time','201').split(',')
        if "capset" in path:
            str_path = ",".join(path)
            path = str_path.replace('capset','126').split(',')
        if "fstat" in path:
            str_path = ",".join(path)
            path = str_path.replace('fstat','5').split(',')
        if "uname" in path:
            str_path = ",".join(path)
            path = str_path.replace('uname','63').split(',')
        if "__NR_vm86old" in path:
            str_path = ",".join(path)
            path = str_path.replace('__NR_vm86old','436').split(',')
        if "__NR_socketcall" in path:
            str_path = ",".join(path)
            path = str_path.replace('__NR_socketcall','437').split(',')
        if "sigaction" in path:
            str_path = ",".join(path)
            path = str_path.replace('sigaction','13').split(',')
        if "ioctl" in path:
            str_path = ",".join(path)
            path = str_path.replace('ioctl','16').split(',')
        if "kmalloc" in path:
            str_path = ",".join(path)
            path = str_path.replace('kmalloc','12').split(',')
        if "shutdown" in path:
            str_path = ",".join(path)
            path = str_path.replace('shutdown','48').split(',')
        if "msgrcv" in path:
            str_path = ",".join(path)
            path = str_path.replace('msgrcv','70').split(',')
        if "ioctl" in path:
            str_path = ",".join(path)
            path = str_path.replace('ioctl','16').split(',')
            
        # 숫자만 남겨두기
        path = re.findall(r'\d+',",".join(path))
        new_pathList.append(path)
    eid_all_path_dict[EID] = new_pathList

In [14]:
# path 가장 긴 길이 찾기
MAX_n = 0
for EID, pathList in eid_all_path_dict.items():
    for path in pathList:
        if MAX_n < len(path):
            MAX_n = len(path)
print(MAX_n)

260


In [15]:
def make_zip(n, path):
    return(zip(*[path[i:] for i in range(n)]))

In [16]:
eid_ngram = dict()
for EID, pathList in eid_all_path_dict.items():
    eid_ngram[EID] = set()
    for path in pathList:
        for n in range(2, MAX_n):
            n_gram = make_zip(n, path)
            if n_gram:
                eid_ngram[EID].update(n_gram)
            else:
                continue

In [17]:
ngram_df = dict()
for EID, ngrams in eid_ngram.items():
    for ngram in ngrams:
        if len(ngram) in ngram_df.keys():
            ngram_df[len(ngram)].add((ngram,EID))
        else:
            ngram_df[len(ngram)] = set()
            ngram_df[len(ngram)].add((ngram,EID))  

In [75]:
import pandas as pd

# 초기화
ngramdf = pd.DataFrame(columns=["N", "Ngram", "exploit"])
for n in range(0, MAX_n):
    tmp = {"N": n, "Ngram":set(), "exploit": set()}
    ngramdf = ngramdf.append(tmp, ignore_index=True)
ngramdf.head()

,N,Ngram,exploit
0,0,{},{}
1,1,{},{}
2,2,{},{}
3,3,{},{}
4,4,{},{}


In [76]:
for N, ngram_EIDs in ngram_df.items():
    for ngram, EID in ngram_EIDs:
        ngramdf.iloc[N]['Ngram'].add(ngram)
        ngramdf.iloc[N]['exploit'].add(EID)
ngramdf.head()

,N,Ngram,exploit
0,0,{},{}
1,1,{},{}
2,2,"{(11, 3), (3, 117), (9, 105), (45, 61), (17, 5...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
3,3,"{(102, 104, 117), (11, 3, 3), (107, 107, 117),...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
4,4,"{(33, 257, 102, 101), (3, 3, 102, 59), (41, 18...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."


In [77]:
ngramdf = ngramdf.drop([0,1])

ngramdf.head()

,N,Ngram,exploit
2,2,"{(11, 3), (3, 117), (9, 105), (45, 61), (17, 5...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
3,3,"{(102, 104, 117), (11, 3, 3), (107, 107, 117),...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
4,4,"{(33, 257, 102, 101), (3, 3, 102, 59), (41, 18...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
5,5,"{(160, 80, 157, 39, 56), (3, 9, 41, 16, 102), ...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
6,6,"{(0, 32, 32, 3, 54, 5), (41, 49, 41, 41, 49, 5...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."


### 양성 어플리케이션 N-gram

In [4]:
import pickle

## Load pickle
with open("seqNumList.pkl","rb") as f:
    seqNumList = pickle.load(f)

In [5]:
app_seq = dict()
for case, seq in seqNumList.items():
    app = case.split("_")[0]
    if app in app_seq.keys():
        app_seq[app].add(tuple(seq))
    else:
        app_seq[app] = set()
        app_seq[app].add(tuple(seq))

In [6]:
import pickle

# pickle로 저장
with open("app_seq.pkl","wb") as f:
    pickle.dump(app_seq, f)

In [7]:
import pickle

# pickle load
with open("app_seq.pkl","rb") as f:
    app_seq = pickle.load(f)

In [8]:
# path 가장 긴 길이 찾기
MAX_app_n = 0
for app, pathList in app_seq.items():
    for path in pathList:
        if MAX_app_n < len(path):
            MAX_app_n = len(path)
print(MAX_app_n)

84921


### 악성 N-gram - 양성 시스템콜 시퀀스

In [52]:
del_Ngram = dict()
for N, row in ngramdf.iterrows():
    del_Ngram[N] = set()
    for ngram in row.Ngram:
        if ngram in del_Ngram[N]:
            continue
        for app, pathSet in app_seq.items():
            if ngram in del_Ngram[N]:
                break
            for path in pathSet:
                if ",".join(ngram) in ","+",".join(path)+",":
                    del_Ngram[N].add(ngram)
                    break

In [72]:
ngramdf.head()

,N,Ngram,exploit
2,2,"{(11, 3), (3, 117), (9, 105), (45, 61), (17, 5...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
3,3,"{(102, 104, 117), (11, 3, 3), (107, 107, 117),...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
4,4,"{(33, 257, 102, 101), (3, 3, 102, 59), (41, 18...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
5,5,"{(160, 80, 157, 39, 56), (3, 9, 41, 16, 102), ...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."
6,6,"{(0, 32, 32, 3, 54, 5), (41, 49, 41, 41, 49, 5...","{6851, 24555, 12, 895, 33523, 9542, 40871, 159..."


In [73]:
ngramdf.iloc[2]['N']

4

In [78]:
# 악성 N-gram에서 제거해야 할 N-gram 제거하기

for index, row in ngramdf.iterrows():
    del_ngram_set = del_Ngram[index]
    origin_ngram_set = row.Ngram
    deleted_ngram_set = origin_ngram_set - del_ngram_set
    ngramdf.iloc[index-2]['Ngram'] = deleted_ngram_set
    ngramdf.iloc[index-2]['exploit'] = set()

# ngramdf.head()

824
552
1432
1258
1719
1610
1860
1795
1954
1912
2015
1990
2064
2048
2114
2100
2135
2122
2129
2117
2100
2089
2050
2040
1980
1972
1902
1895
1826
1821
1755
1750
1698
1694
1656
1653
1634
1631
1619
1616
1612
1609
1613
1610
1617
1614
1615
1612
1610
1607
1603
1600
1594
1591
1585
1582
1569
1566
1552
1549
1547
1544
1548
1545
1554
1551
1571
1568
1601
1598
1644
1641
1690
1687
1738
1735
1786
1783
1832
1829
1867
1864
1890
1887
1896
1893
1888
1885
1871
1868
1846
1843
1815
1812
1779
1776
1734
1731
1683
1680
1624
1621
1552
1549
1463
1460
1362
1359
1255
1252
1146
1143
1039
1036
938
935
851
848
783
780
731
728
692
689
664
661
645
642
634
632
626
624
620
618
614
612
608
606
602
600
596
594
590
588
584
582
578
576
572
570
566
564
560
558
555
553
550
548
545
543
541
540
537
536
534
533
531
530
526
525
521
520
516
515
511
510
506
505
501
500
495
494
488
488
481
481
474
474
467
467
461
461
455
455
448
448
442
442
435
435
428
428
421
421
414
414
408
408
402
402
396
396
390
390
384
384
378
378
372
372
366
366


In [80]:
# pickle로 저장
import pickle

with open("deleted_ngramdf.pkl","wb") as f:
    pickle.dump(ngramdf, f)

ngramdf.head()

,N,Ngram,exploit
2,2,"{(45, 61), (3, 117), (9, 105), (17, 56), (257,...",{}
3,3,"{(49, 42, 0), (102, 104, 117), (0, 0, 68), (77...",{}
4,4,"{(33, 257, 102, 101), (3, 3, 102, 59), (14, 13...",{}
5,5,"{(9, 257, 0, 102, 59), (22, 72, 56, 101, 12), ...",{}
6,6,"{(0, 32, 32, 3, 54, 5), (41, 49, 41, 41, 49, 5...",{}


In [25]:
# pickle load
import pickle

with open("deleted_ngramdf.pkl","rb") as f:
    deleted_ngramdf = pickle.load(f)

deleted_ngramdf.head()

,N,Ngram,exploit
2,2,"{(165, 90), (50, 41), (247, 247), (160, 257), ...",{}
3,3,"{(9, 56, 247), (62, 56, 80), (3, 54, 5), (62, ...",{}
4,4,"{(37, 9, 56, 34), (257, 0, 3, 12), (203, 41, 5...",{}
5,5,"{(13, 213, 53, 44, 53), (294, 12, 12, 254, 254...",{}
6,6,"{(42, 0, 3, 5, 9, 10), (9, 9, 3, 10, 11, 257),...",{}


In [27]:
# ngram policy에 의해 차단되는 exploit 찾기
for index, row in deleted_ngramdf.iterrows():
    for ngram in row.Ngram:
        for eid, pathList in eid_all_path_dict.items():
            for path in pathList:
                if path:
                    if ",".join(ngram) in ","+ ",".join(path) + ",":
                        deleted_ngramdf.iloc[index-2]['exploit'].add(eid)
                        break

deleted_ngramdf.head()

,N,Ngram,exploit
2,2,"{(165, 90), (50, 41), (247, 247), (160, 257), ...","{42276, 895, 15916, 9083, 26131, 12, 10613, 33..."
3,3,"{(9, 56, 247), (62, 56, 80), (3, 54, 5), (62, ...","{42276, 15916, 895, 9083, 26131, 12, 10613, 33..."
4,4,"{(37, 9, 56, 34), (257, 0, 3, 12), (203, 41, 5...","{42276, 895, 15916, 9083, 26131, 12, 10613, 33..."
5,5,"{(13, 213, 53, 44, 53), (294, 12, 12, 254, 254...","{42276, 895, 15916, 9083, 26131, 12, 10613, 33..."
6,6,"{(42, 0, 3, 5, 9, 10), (9, 9, 3, 10, 11, 257),...","{42276, 895, 15916, 9083, 26131, 12, 10613, 33..."


In [32]:
Ngram_cnt = list()
exploit_cnt = list()

for index, row in deleted_ngramdf.iterrows():
    Ngram_cnt.append(len(row.Ngram))
    exploit_cnt.append(len(row.exploit))
    
deleted_ngramdf.insert(2,'Ngram_cnt', Ngram_cnt)
deleted_ngramdf.insert(4,'exploit_cnt', exploit_cnt)

deleted_ngramdf.head()

,N,Ngram,Ngram_cnt,exploit,exploit_cnt
2,2,"{(165, 90), (50, 41), (247, 247), (160, 257), ...",552,"{42276, 895, 15916, 9083, 26131, 12, 10613, 33...",102
3,3,"{(9, 56, 247), (62, 56, 80), (3, 54, 5), (62, ...",1258,"{42276, 15916, 895, 9083, 26131, 12, 10613, 33...",104
4,4,"{(37, 9, 56, 34), (257, 0, 3, 12), (203, 41, 5...",1610,"{42276, 895, 15916, 9083, 26131, 12, 10613, 33...",104
5,5,"{(13, 213, 53, 44, 53), (294, 12, 12, 254, 254...",1795,"{42276, 895, 15916, 9083, 26131, 12, 10613, 33...",101
6,6,"{(42, 0, 3, 5, 9, 10), (9, 9, 3, 10, 11, 257),...",1912,"{42276, 895, 15916, 9083, 26131, 12, 10613, 33...",98


In [33]:
# deleted_ngramdf xlsx 파일로 출력
deleted_ngramdf.to_excel("deleted_ngram_exploit_cnt.xlsx")

In [39]:
# N-gram 개별 exploit 조사
import pandas as pd

two_gramdf = pd.DataFrame(columns=["Ngram", "exploit","exploit_cnt"])

for index, row in deleted_ngramdf.iterrows():
    for ngram in row.Ngram:
        two_gram_dict = {"Ngram": ngram, "exploit": set(), "exploit_cnt": 0}
        for eid, pathList in eid_all_path_dict.items():
            for path in pathList:
                if path:
                    if ",".join(ngram) in ","+ ",".join(path) + ",":
                        two_gram_dict["exploit"].add(eid)
                        break
        two_gram_dict["exploit_cnt"] = len(two_gram_dict["exploit"])
        two_gramdf = two_gramdf.append(two_gram_dict, ignore_index=True)
two_gramdf.head()

,Ngram,exploit,exploit_cnt
0,"(165, 90)","{37292, 39166}",2
1,"(50, 41)",{41458},1
2,"(247, 247)",{43127},1
3,"(160, 257)",{42887},1
4,"(56, 140)",{2013},1


In [49]:
# 2-gram exploit 조사
import pandas as pd

two_gram = pd.DataFrame(columns=["Ngram", "exploit","exploit_cnt"])

for index, row in two_gramdf.iterrows():
    if len(row.Ngram) == 2:
        two_gram_dict = {"Ngram":row.Ngram, "exploit": row.exploit, "exploit_cnt":row.exploit_cnt}
        two_gram = two_gram.append(two_gram_dict, ignore_index=True)
two_gram.head()

,Ngram,exploit,exploit_cnt
0,"(165, 90)","{37292, 39166}",2
1,"(50, 41)",{41458},1
2,"(247, 247)",{43127},1
3,"(160, 257)",{42887},1
4,"(56, 140)",{2013},1


In [53]:
# Total = 106

ex = set()
for index, row in two_gramdf.iterrows():
    if len(ex) == 106:
        print(len(row.Ngram))
        print(row.Ngram)
    ex.update(row.exploit)

len(ex)

104

In [57]:
two_gram = two_gram.sort_values('exploit_cnt', ascending=False)
two_gram.head()

,Ngram,exploit,exploit_cnt
338,"(56, 230)","{44300, 17787, 2005, 25288, 15285, 15704, 3134...",16
98,"(56, 10)","{18411, 33824, 35161, 50541, 15944, 8678, 4460...",14
209,"(102, 59)","{44300, 17787, 43029, 15285, 15774, 15704, 400...",14
549,"(117, 119)","{1397, 33321, 47170, 10613, 33516, 14814, 5093...",13
386,"(101, 101)","{35161, 50541, 8678, 22363, 47163, 22362, 3, 2...",12


In [59]:
policy_2gram = {"Ngram":set(), "exploit":set()}
for index, row in two_gram.iterrows():
    if len(policy_2gram["exploit"] | row.exploit) > len(policy_2gram["exploit"]):
        policy_2gram["Ngram"].add(row.Ngram)
        policy_2gram["exploit"].update(row.exploit)
    
print("Ngram #: ",len(policy_2gram["Ngram"]))
print("Exploit #: ",len(policy_2gram["exploit"]))

Ngram #:  31
Exploit #:  102


In [60]:
# 3-gram exploit 조사
import pandas as pd

three_gram = pd.DataFrame(columns=["Ngram", "exploit","exploit_cnt"])

for index, row in two_gramdf.iterrows():
    if len(row.Ngram) == 3:
        three_gram_dict = {"Ngram":row.Ngram, "exploit": row.exploit, "exploit_cnt":row.exploit_cnt}
        three_gram = three_gram.append(three_gram_dict, ignore_index=True)
three_gram.head()

,Ngram,exploit,exploit_cnt
0,"(9, 56, 247)",{43029},1
1,"(62, 56, 80)",{2031},1
2,"(3, 54, 5)",{45553},1
3,"(62, 230, 257)","{2006, 2031}",2
4,"(0, 5, 0)","{43029, 15916, 895, 9083, 33336, 12, 1397, 335...",28


In [61]:
three_gram = three_gram.sort_values('exploit_cnt', ascending=False)
three_gram.head()

,Ngram,exploit,exploit_cnt
64,"(14, 56, 11)","{43127, 44302, 44298, 926, 33336, 37292, 50808...",29
800,"(56, 11, 14)","{43127, 44302, 44298, 926, 33336, 37292, 50808...",29
758,"(13, 14, 9)","{43127, 44302, 44298, 926, 33336, 37292, 50808...",29
482,"(14, 9, 14)","{43127, 44302, 44298, 926, 33336, 37292, 50808...",29
821,"(14, 61, 13)","{43127, 44302, 44298, 926, 33336, 37292, 50808...",29


In [62]:
policy_2_3_gram = policy_2gram
for index, row in three_gram.iterrows():
    if len(policy_2_3_gram["exploit"] | row.exploit) > len(policy_2_3_gram["exploit"]):
        policy_2_3_gram["Ngram"].add(row.Ngram)
        policy_2_3_gram["exploit"].update(row.exploit)
    
print("Ngram #: ",len(policy_2_3_gram["Ngram"]))
print("Exploit #: ",len(policy_2_3_gram["exploit"]))

Ngram #:  33
Exploit #:  104


In [50]:
# Excel에 저장
two_gram.sort_values('exploit_cnt', ascending=False)

,Ngram,exploit,exploit_cnt
338,"(56, 230)","{44300, 17787, 2005, 25288, 15285, 15704, 3134...",16
98,"(56, 10)","{18411, 33824, 35161, 50541, 15944, 8678, 4460...",14
209,"(102, 59)","{44300, 17787, 43029, 15285, 15774, 15704, 400...",14
549,"(117, 119)","{1397, 33321, 47170, 10613, 33516, 14814, 5093...",13
386,"(101, 101)","{35161, 50541, 8678, 22363, 47163, 22362, 3, 2...",12
...,...,...,...
319,"(230, 54)",{41995},1
318,"(186, 56)",{35370},1
121,"(35, 11)",{40871},1
122,"(3, 45)",{41458},1
